In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import matplotlib.pyplot as plt

from numpy.random import seed

In [ ]:
train = pd.read_csv("/kaggle/input/titanic/train.csv")

train = train.drop(columns=['Name', 'Cabin', 'Ticket'])

train['Family'] = train['SibSp'] + train['Parch']

train = train.drop(columns=['SibSp', 'Parch'])

print(train.columns)

train['Age'] = train['Age'].interpolate()
train['Embarked'] = train['Embarked'].interpolate()

train = train.sample(frac=1)

train.info()

In [ ]:
# subset all categorical variables which need to be encoded
categorical = ['Embarked']

for var in categorical:
    train = pd.concat([train, 
                    pd.get_dummies(train[var], prefix=var)], axis=1)
    del train[var]
    
train.info()

In [ ]:
# converting to values
# train['Embarked'] = train['Embarked'].replace(to_replace = 'C', value = 1)
# train['Embarked'] = train['Embarked'].replace(to_replace = 'S', value = 2)
# train['Embarked'] = train['Embarked'].replace(to_replace = 'Q', value = 3)

# coverting to values
train['Sex'] = train['Sex'].replace(to_replace = 'female', value = 0)
train['Sex'] = train['Sex'].replace(to_replace = 'male', value = 1)

In [ ]:
continuous = ['Age', 'Fare', 'Pclass', 'Family', 'Sex']

scaler = StandardScaler()

for var in continuous:
    train[var] = train[var].astype('float64')
    train[var] = scaler.fit_transform(train[var].values.reshape(-1, 1))

train

In [ ]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)
        
display_all(train.describe(include='all').T)

In [ ]:
train_inputs = train.drop(columns=['Survived', 'PassengerId'])
train_labels = train['Survived']

train_len = 800
dev_inputs = train_inputs[train_len:]
dev_labels = train_labels[train_len:]

train_inputs = train_inputs[:train_len]
train_labels = train_labels[:train_len]

print(train_inputs.shape)
print(train_labels.shape)
print(dev_inputs.shape)
print(dev_labels.shape)

In [ ]:
seed(42)
tf.random.set_seed(42)

model = tf.keras.Sequential([
	tf.keras.layers.Dense(units=10, activation="relu",input_shape=(8,)),
    tf.keras.layers.Dense(units=6, activation='relu'),
	tf.keras.layers.Dense(units=1, activation="sigmoid"),
])

model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_inputs, train_labels, epochs=500, batch_size=128, validation_data=(dev_inputs, dev_labels))

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
# summarize history for accuracy
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
test = pd.read_csv("/kaggle/input/titanic/test.csv")

test = test.drop(columns=['Name', 'Cabin', 'Ticket'])

test['Age'] = test['Age'].interpolate()
test['Embarked'] = test['Embarked'].interpolate()

test.info()

In [ ]:
# subset all categorical variables which need to be encoded
print(test.columns)

categorical = ['Embarked']

for var in categorical:
    test = pd.concat([test, 
                    pd.get_dummies(test[var], prefix=var)], axis=1)
    del test[var]
    
test.info()

In [ ]:

# coverting to values
test['Sex'] = test['Sex'].replace(to_replace = 'female', value = 0)
test['Sex'] = test['Sex'].replace(to_replace = 'male', value = 1)

test['Family'] = test['SibSp'] + test['Parch']

test = test.drop(columns=['SibSp', 'Parch'])

In [ ]:
continuous = ['Age', 'Fare', 'Pclass', 'Family', 'Sex']

scaler = StandardScaler()

for var in continuous:
    test[var] = test[var].astype('float64')
    test[var] = scaler.fit_transform(test[var].values.reshape(-1, 1))

test

In [ ]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)
        
display_all(test.describe(include='all').T)

In [ ]:
test

In [ ]:
test['Fare'] = test['Fare'].interpolate()

ans = model.predict(test.drop(columns=['PassengerId']))

final_ans = []
for i in ans:
    if i < 0.5:
        final_ans.append(0)
    else:
        final_ans.append(1)

ans_df = pd.DataFrame(final_ans, columns=['Survived'])
print(ans_df.shape)
print(test['PassengerId'].shape)
pass_id = test['PassengerId']
result = pd.concat([pass_id, ans_df], axis=1, sort=False)
result

In [ ]:
result.to_csv('submission.csv')